CONTINUE TO TEST REGULARIZATION
TEST EFFECTS OF CHANGING TO IMAGENET

In [1]:
import torchvision 
import torch
from torch.utils.data import Dataset
from torchvision import datasets
import torchvision.transforms as transforms   
import torch.optim as optim
from torchvision.transforms import RandomRotation
from torchvision.transforms import Pad
from torchvision.transforms import Resize
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from PIL import Image
from numpy.linalg import norm
from torch.cuda.random import device_count
from torchvision.datasets import CIFAR10
from torch.utils.data import random_split
import torch.optim as optim
import torchvision.transforms as transforms 
import torchvision 
import math
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torch.nn.utils.prune as prune
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.autograd import Variable
#!pip install opacus


In [2]:
import scipy
mat = scipy.io.loadmat('/notebooks/data_cifar100_train.mat')
mat2 = scipy.io.loadmat('/notebooks/data_cifar100_test.mat')
lab = scipy.io.loadmat('/notebooks/label_train.mat')
lab2 = scipy.io.loadmat('/notebooks/label_test.mat')

data = mat['data'] # use the key for data here
target = lab['data']

data = torch.from_numpy(data).float()
target = torch.from_numpy(target).long() # change type to your use case
target = torch.flatten(target)

data2 = mat2['data'] # use the key for data here
target2 = lab2['data']

data2 = torch.from_numpy(data2).float()
target2 = torch.from_numpy(target2).long() # change type to your use case
target2 = torch.flatten(target2)
dataset = torch.utils.data.TensorDataset(data, target)
testset = torch.utils.data.TensorDataset(data2, target2)

indices = list(range(0,50000,1))
trainset1 = torch.utils.data.Subset(dataset, indices)
modelloader = torch.utils.data.DataLoader(trainset1, shuffle=True, num_workers=2, batch_size = 256)

valid = list(range(40000, 50000,1))
validset1 = torch.utils.data.Subset(dataset, valid)
validloader = torch.utils.data.DataLoader(validset1, shuffle=True, num_workers=2,batch_size = 100)
testloader = torch.utils.data.DataLoader(testset, shuffle=True, num_workers=2, batch_size = 100)

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']


In [3]:
from torch.cuda.random import device_count
from torchvision.datasets import CIFAR10
from torch.utils.data import random_split
import torch.optim as optim
import torchvision.transforms as transforms 
import torchvision 

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
transform1 = transforms.Compose(
    [    
    transforms.RandomCrop(32, 4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandAugment(num_ops=2, magnitude=12),
    transforms.ColorJitter(0.3, 0.3, 0.3),
    transforms.ToTensor(),
    transforms.Normalize((0.4, 0.4822, 0.4465), (0.247, 0.243, 0.261)),   
    ])
trainset = torchvision.datasets.CIFAR10(root='./', train=True,download=True,transform = transform1)
indices = list(range(0, 40000,1))
trainset1 = torch.utils.data.Subset(trainset, indices)
modelloader = torch.utils.data.DataLoader(trainset1, shuffle=True, num_workers=2, batch_size = 5000)
validset = torchvision.datasets.CIFAR10(root='./', train=True,download=True, transform = transform) 
valid = list(range(40000, 50000,1))
validset1 = torch.utils.data.Subset(validset, valid)
validloader = torch.utils.data.DataLoader(validset1, shuffle=True, num_workers=2,batch_size = 100)
testset = torchvision.datasets.CIFAR10(root='./data', train = False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, shuffle=True, num_workers=2, batch_size = 1)
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        x = x.view(-1, 1024)
        out = self.linear(x)
        return out

In [7]:

def standardize(x, bn_stats):
    if bn_stats is None:
        return x

    bn_mean, bn_var = bn_stats

    view = [1] * len(x.shape)
    view[1] = -1
    x = (x - bn_mean.view(view)) / torch.sqrt(bn_var.view(view) + 1e-5)

    # if variance is too low, just ignore
    x *= (bn_var.view(view) != 0).float()
    return x

class StandardizeLayer(nn.Module):
    def __init__(self, bn_stats):
        super(StandardizeLayer, self).__init__()
        self.bn_stats = bn_stats

    def forward(self, x):
        return standardize(x, self.bn_stats)


In [8]:
#from opacus import PrivacyEngine
inputDim = 1024 # takes variable 'x' 
outputDim = 10       # takes variable 'y'
learningRate = 0.005
epochs = 100

model = nn.Sequential( nn.Linear(1024, 10)).to(device)
##### For GPU #######
if torch.cuda.is_available():
    model.cuda()
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1,momentum = 0.9, nesterov= True)
#torch.optim.SGD(model.parameters(), lr=learningRate, momentum = 0.9, weight_decay = 0.001)
"""
privacy_engine = PrivacyEngine()

model, optimizer, modelloader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=modelloader,
    noise_multiplier=1.0,
    max_grad_norm=0.1,
)
"""

'\nprivacy_engine = PrivacyEngine()\n\nmodel, optimizer, modelloader = privacy_engine.make_private(\n    module=model,\n    optimizer=optimizer,\n    data_loader=modelloader,\n    noise_multiplier=1.0,\n    max_grad_norm=0.1,\n)\n'

In [10]:
from numpy.linalg import norm


train_losslist = []
train_losslist1 = []
#number of epochs to train the model
n_epochs = [*range(100)] # you may increase this number to train a final model

valid_loss_min = np.Inf # track change in validation loss
for epoch in n_epochs:
    # keep track of training and validation loss
    train_loss = 0.0
    train_loss1 = 0.0
    valid_loss = 0.0   
    ###################
    # train the model #
    ###################
    model.train()
    model.to(device)
    #model1.train()
    class_correct1 = list(0. for i in range(10))
    class_total1 = list(0. for i in range(10))
    for data, target in modelloader:
        optimizer.zero_grad()
        data, target = data.to(device), target.to(device)
        output = model(data)
        #loss = criterion(output, target)
        loss = F.cross_entropy(output,target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        #train_loss1 += loss1.item()*data.size(0)
        _, pred = torch.max(output, 1)  
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(target.size(dim=0)):
            label = target.data[i]
            class_correct1[label] += correct[i].item()
            class_total1[label] += 1
            
    #print(privacy_engine.get_epsilon(delta=5e-05))
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    for data, target in testloader:
        data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)  
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(100):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
            # calculate average losses
    valid_loss = valid_loss/len(validloader.dataset)
    train_loss = train_loss/len(modelloader.dataset)
    train_losslist.append(train_loss)
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    print('Training Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct1) / np.sum(class_total1),
    np.sum(class_correct1), np.sum(class_total1)))
    print('Validation Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))
#print(privacy_engine.get_epsilon(delta=5e-05))
#torch.save(model.state_dict(), "10000top")           

Epoch: 0 	Training Loss: 0.602179 	Validation Loss: 0.610484
Training Accuracy (Overall): 78% (39205/50000)
Validation Accuracy (Overall): 78% (7829/10000)
Epoch: 1 	Training Loss: 0.583976 	Validation Loss: 0.590599
Training Accuracy (Overall): 79% (39515/50000)
Validation Accuracy (Overall): 78% (7894/10000)
Epoch: 2 	Training Loss: 0.573256 	Validation Loss: 0.609558
Training Accuracy (Overall): 79% (39787/50000)
Validation Accuracy (Overall): 78% (7808/10000)
Epoch: 3 	Training Loss: 0.562763 	Validation Loss: 0.585721
Training Accuracy (Overall): 79% (39925/50000)
Validation Accuracy (Overall): 79% (7960/10000)
Epoch: 4 	Training Loss: 0.556070 	Validation Loss: 0.569016
Training Accuracy (Overall): 80% (40053/50000)
Validation Accuracy (Overall): 80% (8015/10000)
Epoch: 5 	Training Loss: 0.548447 	Validation Loss: 0.567646
Training Accuracy (Overall): 80% (40251/50000)
Validation Accuracy (Overall): 80% (8026/10000)
Epoch: 6 	Training Loss: 0.542820 	Validation Loss: 0.589009
Tra

KeyboardInterrupt: 

full batch
10000: 55 epsilon, 62%
15000: 50 epsilon, 62%
20000: 50 epsilon, 62%
25000: 50 epsilon, 63%
30000: 50 epsilon, 61%
35000: 50 epsilon, 63%

5k batch
10000:50 epsilon, 66%
15000: 50 epsilon, 68%

10k batch
20000: 50, 66%
25000: 50, 68%

adam
5k batch
10000: 76%, 50 epsilon
15000: 77%, 50 epsilon

10k batch
20000: 77, 50 epsilon







In [11]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
test_loss = 0.
train_on_gpu = torch.cuda.is_available()
model.eval()
# iterate over test data
for data, target in testloader:
    # move tensors to GPU if CUDA is available
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(100):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
# average test loss
test_loss = test_loss/len(modelloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
          classes[i], 100 * class_correct[i] / class_total[i],
          np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))
print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
100. * np.sum(class_correct) / np.sum(class_total),
np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.096982

Test Accuracy of airplane: 86% (869/1000)
Test Accuracy of automobile: 94% (945/1000)
Test Accuracy of  bird: 75% (757/1000)
Test Accuracy of   cat: 64% (643/1000)
Test Accuracy of  deer: 84% (848/1000)
Test Accuracy of   dog: 74% (747/1000)
Test Accuracy of  frog: 90% (900/1000)
Test Accuracy of horse: 85% (852/1000)
Test Accuracy of  ship: 89% (896/1000)
Test Accuracy of truck: 84% (848/1000)

Test Accuracy (Overall): 83% (8305/10000)


In [3]:
matrix = np.array([])
count = 1
avg = 0 
topacc = 0
for i in range(600): 
    print("##########################################################")
    print("#########################################################")
    print("trial: " + str(i))
    indices = torch.randperm(len(dataset))[:1000]
    #indices = list(range(0, 10000,1))
    trainset1 = torch.utils.data.Subset(dataset, indices)
    modelloader = torch.utils.data.DataLoader(trainset1, shuffle=True, num_workers=2, batch_size = 1000)
    testloader = torch.utils.data.DataLoader(testset, shuffle=True, num_workers=2, batch_size = 100)
    #model pretrained by resnet
    #model = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)
    model = nn.Sequential( nn.Linear(1024, 10)).to(device)
    model.load_state_dict(torch.load("/notebooks/start"))
    model.to(device) 
    #print(get_n_params(model))
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.01,momentum = 0.9, nesterov = True)
    #torch.optim.Adam(model.parameters(), lr = 0.01)
     #torch.optim.SGD(model.parameters(), lr = 0.01,momentum = 0.9, nesterov= True)
    
  ################
  #training model
  ####################
    train_losslist = []
    n_epochs = [*range(50)]
    
    valid_loss_min = np.Inf # track change in validation loss

    for epoch in n_epochs:
    # keep track of training and validation loss
        train_loss = 0.0
        train_loss1 = 0.0
        valid_loss = 0.0   
    ###################
    # train the model #
    ###################
        model.train()
    #model1.train()
        class_correct1 = list(0. for i in range(10))
        class_total1 = list(0. for i in range(10))
        for data, target in modelloader:
        # move tensors to GPU if CUDA is available
            data, target = data.to(device), target.to(device)
        # clear the gradients of all optimized variables
            optimizer.zero_grad()
        #optimizer1.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
        #output1 = model1(data)
        # calculate the batch loss
            loss = criterion(output, target)
        #loss1 = criterion1(output1,target)
        # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
        #loss1.backward()
        # perform a single optimization step (parameter update)
            optimizer.step()
        #exp_lr_scheduler.step()
        #optimizer1.step()
        # update training loss
            train_loss += loss.item()*data.size(0)
        #train_loss1 += loss1.item()*data.size(0)
            _, pred = torch.max(output, 1)    
        # compare predictions to true label
            correct_tensor = pred.eq(target.data.view_as(pred))
            correct = np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
            for i in range(100):
                label = target.data[i]
                class_correct1[label] += correct[i].item()
                class_total1[label] += 1

    ########################
    # testing model #
    ######################### 
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    test_loss = 0.
    train_on_gpu = torch.cuda.is_available()
    model.eval()
    # iterate over test data
    for data, target in testloader:
        # move tensors to GPU if CUDA is available
        data, target = data.to(device), target.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        loss = criterion(output, target)
        # update test loss 
        test_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)    
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(100):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
    # average test loss
    test_loss = test_loss/len(modelloader.dataset)
    print('Test Loss: {:.6f}\n'.format(test_loss))

    for i in range(10):
        if class_total[i] > 0:
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
              classes[i], 100 * class_correct[i] / class_total[i],
              np.sum(class_correct[i]), np.sum(class_total[i])))
        else:(' uracy of %5s: N/A (no training examples)' % (classes[i]))
    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))        
    avg += (100. * np.sum(class_correct) / np.sum(class_total))
    if(np.sum(class_correct) / np.sum(class_total)*100. > topacc):
        topacc = np.sum(class_correct) / np.sum(class_total)*100.
        torch.save(model.state_dict(), "50epoch1ktop")   
    arr = np.array([])
    for name, param in model.named_parameters():
        temp = param.cpu().detach().numpy()
        bruh = temp.flatten()
        arr =np.concatenate((arr,bruh),axis = 0)
    arr =np.reshape(arr, (1,-1))
    if matrix.size == 0:
        matrix = arr
    else: 
        matrix = np.append(matrix, arr, axis = 0)
    count = count +1
print(avg/300)
np.save('600iter1k50epochnesterov.npy', matrix)

##########################################################
#########################################################
trial: 0
Test Loss: 10.674662

Test Accuracy of airplane: 66% (663/1000)
Test Accuracy of automobile: 85% (859/1000)
Test Accuracy of  bird: 46% (465/1000)
Test Accuracy of   cat: 39% (397/1000)
Test Accuracy of  deer: 59% (592/1000)
Test Accuracy of   dog: 76% (761/1000)
Test Accuracy of  frog: 89% (891/1000)
Test Accuracy of horse: 71% (718/1000)
Test Accuracy of  ship: 77% (771/1000)
Test Accuracy of truck: 73% (739/1000)

Test Accuracy (Overall): 68% (6856/10000)
##########################################################
#########################################################
trial: 1
Test Loss: 10.681574

Test Accuracy of airplane: 75% (750/1000)
Test Accuracy of automobile: 83% (831/1000)
Test Accuracy of  bird: 45% (450/1000)
Test Accuracy of   cat: 54% (543/1000)
Test Accuracy of  deer: 68% (681/1000)
Test Accuracy of   dog: 69% (696/1000)
Test Accuracy of  fro

In [3]:

def l2norm(data,size): 
    matrix = data
    count = 0
    total = 0
    for i in range(size):
        for j in range(i+1, size):
            data1 = matrix[i]
            data2 = matrix[j]
            difference = data1 -data2
            total += np.linalg.norm(difference)
            
            count = count+1
    print(total/count)
    print(np.linalg.norm(matrix[0]))
    print((total/count)/np.linalg.norm(matrix[0]))
def l2normupdated(data,size):
    count = 0
    total = 0
    avg = np.array([])
    for i in range(size):
        if i ==0:
            avg = data[i]
        else:
            avg = avg+data[i]
    avg = avg/size
    print(avg)
    print(data[0])
    for i in range(size):
        data1 = avg-data[i]
        total += np.linalg.norm(data1)
    print(total/size)
    print(np.linalg.norm(avg))
    print((total/size)/np.linalg.norm(avg))
    return total/size


def test_model(model):
    test_loss = 0.0
    testloader = torch.utils.data.DataLoader(testset, shuffle=True, num_workers=2, batch_size = 20)
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    train_on_gpu = torch.cuda.is_available()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    model.eval()
    # iterate over test data
    for data, target in testloader:
        # move tensors to GPU if CUDA is available
        data, target = data.to(device), target.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update test loss 
        test_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)    
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(20):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
    # average test loss
    test_loss = test_loss/len(testloader.dataset)
    #Q print('Test Loss: {:.6f}\n'.format(test_loss))
    
    for i in range(10):
        if class_total[i] > 0:
            
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
              classes[i], 100 * class_correct[i] / class_total[i],
              np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))
    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))
    return np.sum(class_correct) / np.sum(class_total)

In [4]:



matrix = np.load('/notebooks/30010k1pointremoval100epoch0.1lropacus.npy', allow_pickle = True)
matrix2 = np.load('/notebooks/300iter5krandom1.npy',allow_pickle = True)
matrix3 = np.load('/notebooks/300iter5krandom2.npy',allow_pickle = True)
matrix4 = np.append(matrix2,matrix3,axis = 0)
matrix5 = np.append(matrix,matrix4,axis = 0)
print(np.shape(matrix))
data = torch.from_numpy(matrix)
print(data)

print(f"input matrix successfully loaded. It is of size {data.size()}")
"""
model = nn.Sequential( nn.Linear(1024, 10)).to(device)
noise = data[0]
for param in model.parameters():
    needed = param.numel()
    use, left = torch.split(noise, [needed, noise.numel()-needed])
    use1 =torch.reshape(use,param.size())
    noise1 = torch.nn.Parameter(use1.cpu()).to(device)
    with torch.no_grad():
        param.copy_(noise1)
    noise = left
test_model(model)
torch.save(model.state_dict(), "10000top")  
"""
"""
eig = eig_estimation(data,10)
print(eig)
print(eig.size())
print(torch.nansum(torch.sqrt(eig)))
"""
l2norm(matrix,2)
l2normupdated(matrix,2)
"""
eig = get_sample_covariance_eigs(data)

print(eig)

print(eig.size())
print(torch.nansum(torch.sqrt(eig)))
"""

#0.03721526866805743


(100, 10250)
tensor([[ 0.0184, -0.0136,  0.0089,  ..., -0.0213,  0.0343,  0.0074],
        [ 0.0186, -0.0140,  0.0088,  ..., -0.0213,  0.0343,  0.0074],
        [ 0.0187, -0.0141,  0.0089,  ..., -0.0215,  0.0344,  0.0074],
        ...,
        [ 0.0185, -0.0139,  0.0088,  ..., -0.0214,  0.0342,  0.0074],
        [ 0.0184, -0.0138,  0.0089,  ..., -0.0213,  0.0343,  0.0074],
        [ 0.0186, -0.0138,  0.0089,  ..., -0.0213,  0.0343,  0.0072]],
       dtype=torch.float64)
input matrix successfully loaded. It is of size torch.Size([100, 10250])
0.013476521926138841
2.0099143430782296
0.006705023013816221
[ 0.01849803 -0.01379479  0.0088659  ... -0.02131343  0.03430561
  0.0073958 ]
[ 0.01844439 -0.01362507  0.00891462 ... -0.02128817  0.03434535
  0.00737524]
0.0067382609630694205
2.009851625531255
0.0033526161222415243


'\neig = get_sample_covariance_eigs(data)\n\nprint(eig)\n\nprint(eig.size())\nprint(torch.nansum(torch.sqrt(eig)))\n'

1k random: 0.82

2.5k random: 0.51

5k random: 0.35

7.5k random: 0.28

10k random: 0.239

1k removal: 0.025

1k 0.1 lr 50 epoch: 0.067

1k 0.1 lr 100 epoch: 0.096

1k 0.1 lr 100epoch 0.25grad: 0.01

10k removal: 0.002

10k 0.1 lr 50 epoch: 0.007

10k 0.1 lr 100 epoch:0.011

10k 0.1 lr 100 epoch 0.25grad: 0.009

10k 0.05 lr 100 epoch 0.25grad: 0.001

10k 0.1 lr 100 epoch 0.05grad: 0.004

10k 0.1 lr 200 epoch 0.05grad: 0.012

10k 0.1 lr 100 epoch 10 point: 0.04

10k 0.1 lr 100 epoch 100 point: 0.138

10k 0.1 lr 100 epoch 1000 point: 0.459

10k 0.1 lr 100 epoch 2500 point: 0.795

10k 0.1 lr 100 epoch 5000 point: 1.37


20k 0.1 lr 50 epoch: 0.0004

20k 0.1 lr 100 epoch: 0.005

20k 0.1 lr 100 epoch 0.25grad:0.005

20k 0.1 lr  epoch 0.25grad:0.005




Adam:
1000:
77% l2 norm
44 square root sum 
2500:
55% l2 norm
30 square root sum
5000:
42% l2 norm
21 square root sum
10000:
29% l2 norm
14 square root sum
SGD:
1000:
41% l2 norm
13 square root sum
2500:
26% l2 norm
8 square root sum
5000:
18% l2 norm
5.77
7500:
14% l2 norm
4.6 square root sum
10000:
12% l2 norm
3.8 square root sum
12500:
10% l2 norm
3.3 square root sum
15000:
9% l2 norm
2.97 square root sum
17500:
8% 
2.6 square root sum






In [28]:
def gen_noise2(data,q,noise_mult=0.5):
    U, S, V = torch.pca_lowrank(data, q, center=True, niter=2)
    Sprime = torch.sqrt(S)
    Sprime *= torch.nansum(Sprime)
    Sprime = torch.sqrt(Sprime)
   # cov = torch.cov(data.t())
    print("here")
    noise = torch.randn(data.size()[0], device=device)*noise_mult
    temp = torch.diag(Sprime).type(torch.FloatTensor)
    temp = temp.to(device)
    noise = torch.matmul(noise, temp)
    noise = noise.type(torch.DoubleTensor)
    noise = torch.matmul(V.to(device), noise.to(device))
    print(noise)
    return noise

    
def add_noise2(model, noise):
    model1 = model
    for param in model1.parameters():
        needed = param.numel()
        use, left = torch.split(noise, [needed, noise.numel()-needed])
        use1 =torch.reshape(use,param.size())
        noise1 = torch.nn.Parameter(param.cpu() + use1.cpu()).to(device)
        with torch.no_grad():
            param.copy_(noise1)
        noise = left
    return model1

In [31]:
average1 = 0
average2 = 0
matrix = np.load('/notebooks/30010k1pointremoval100epoch0.1lr.npy', allow_pickle = True)
data = torch.from_numpy(matrix)
for i in range(20):
    print("trial " + str(i))
    data2 = gen_noise2(data,300)
#    print(ga ussian.size())
    model = nn.Sequential( nn.Linear(1024, 10)).to(device)
    model.load_state_dict(torch.load("/notebooks/9999100ep0.1lrtop"))
    model.to(device)
    average1 += test_model(model)
    model1 = add_noise2(model, data2)
    #print(test_model(model))
    average2 += test_model(model1)
print(average1/20)
print(average2/20)


trial 0
here
tensor([-0.0515, -0.3024,  0.1446,  ...,  0.0753,  0.2021, -0.1144],
       device='cuda:0', dtype=torch.float64)
Test Accuracy of airplane: 78% (780/1000)
Test Accuracy of automobile: 88% (885/1000)
Test Accuracy of  bird: 67% (670/1000)
Test Accuracy of   cat: 61% (616/1000)
Test Accuracy of  deer: 72% (721/1000)
Test Accuracy of   dog: 67% (671/1000)
Test Accuracy of  frog: 88% (880/1000)
Test Accuracy of horse: 77% (770/1000)
Test Accuracy of  ship: 85% (859/1000)
Test Accuracy of truck: 85% (858/1000)

Test Accuracy (Overall): 77% (7710/10000)
Test Accuracy of airplane: 51% (510/1000)
Test Accuracy of automobile: 77% (779/1000)
Test Accuracy of  bird: 34% (345/1000)
Test Accuracy of   cat: 40% (409/1000)
Test Accuracy of  deer: 38% (380/1000)
Test Accuracy of   dog: 45% (457/1000)
Test Accuracy of  frog: 60% (604/1000)
Test Accuracy of horse: 37% (375/1000)
Test Accuracy of  ship: 57% (575/1000)
Test Accuracy of truck: 65% (653/1000)

Test Accuracy (Overall): 50% (508

1000 average after noise
41
2500 average
45
5000
49
10000
55????
12500
57
15000
59
17500
50

adam goes up to 68
